Сначала нужно запустить сервер из консоли:
```bash
$ python src/app.py
```

In [1]:
import requests

Создадим банки и клиентов

In [2]:
requests.post('http://localhost:8000/superuser/create_bank', timeout=0.01, json={
    'name': 'bank1'
}).json()

{'message': 'Created bank bank1', 'status': 'ok'}

In [3]:
requests.post('http://localhost:8000/superuser/create_bank', timeout=0.01, json={
    'name': 'bank2'
}).json()

{'message': 'Created bank bank2', 'status': 'ok'}

In [4]:
client1 = requests.post('http://localhost:8000/superuser/create_client', timeout=0.01, json={
    'bank': 'bank1',
    'name': 'Иван',
    'surname': 'Иванов',
    'address': 'Москва, ул. Ленина, д. 1',
    'passport': '1234 567890'
}).json()
client1

{'client_token': '5d5c90af-3b67-4568-91aa-457ea72b6b1e',
 'message': 'Created client',
 'status': 'ok'}

In [5]:
client2 = requests.post('http://localhost:8000/superuser/create_client', timeout=0.01, json={
    'bank': 'bank2',
    'name': 'Петр',
    'surname': 'Петров'
}).json()
client2

{'client_token': 'ee1d7f5e-9aca-4726-9a26-885722df70a7',
 'message': 'Created client',
 'status': 'ok'}

In [6]:
# Обновим данные второго клиента
requests.post('http://localhost:8000/superuser/update_client', timeout=0.01, json={
    'client_token': client2['client_token'],
    'address': 'Москва, ул. Ленина, д. 2',
    'passport': '1234 567891'
}).json()

{'message': "Now client is {'bank': <core.Bank object at 0x101d6ae50>, 'name': 'Петр', 'surname': 'Петров', 'passport': '1234 567891', 'address': 'Москва, ул. Ленина, д. 2', 'accounts': {UUID('c88f3fda-8d1e-421c-b1dc-a11bf9f457cf'): <core.CashAccount object at 0x101d6a4d0>}, 'default_cash_account': <core.CashAccount object at 0x101d6a4d0>}",
 'status': 'ok'}

Создадим счета и внесем наличные

In [7]:
account1 = requests.post('http://localhost:8000/client/create_account', timeout=0.01, json={
    'client_token': client1['client_token'],
    'bank': 'bank1',
    'account_type': 'CreditAccount',
    'kwargs': {'credit_limit': 10_000, 'interest_rate': 0.1},
}).json()
account1

{'info': {'balance': 0,
  'id': '8b4a28f8-1209-4751-976d-db03799da425',
  'type': 'CreditAccount'},
 'message': 'Created account',
 'status': 'ok'}

In [8]:
account2 = requests.post('http://localhost:8000/client/create_account', timeout=0.01, json={
    'client_token': client2['client_token'],
    'bank': 'bank1',
    'account_type': 'DepositAccount',
    'kwargs': {'end_date': '2036-01-01'},
}).json()
account2

{'info': {'balance': 0,
  'id': '3092d856-a0bd-464d-adc0-6c48146ab132',
  'type': 'DepositAccount'},
 'message': 'Created account',
 'status': 'ok'}

In [9]:
requests.post('http://localhost:8000/client/deposit', timeout=0.01, json={
    'client_token': client2['client_token'],
    'account_id': account2['info']['id'],
    'amount': 4000,
}).json()

{'message': 'Deposited 4000', 'status': 'ok'}

Сделаем перевод

In [10]:
requests.post('http://localhost:8000/client/transfer', timeout=0.01, json={
    'client_token': client1['client_token'],
    'to_bank_name': 'bank2',
    'from_account_id': account1['info']['id'],
    'to_account_id': account2['info']['id'],
    'amount': 1000
}).json()

{'message': 'Transferred 1000', 'status': 'ok'}

Попробуем вернуть деньги

In [11]:
requests.post('http://localhost:8000/client/transfer', timeout=0.01, json={
    'client_token': client2['client_token'],
    'to_bank_name': 'bank1',
    'from_account_id': account2['info']['id'],
    'to_account_id': account1['info']['id'],
    'amount': 1000
}).json()

{'message': "Error: Can't withdraw money before end date\n", 'status': 'error'}

Не получилось. Воспользуемся механизмом отката транзакций.

История транзакций:

In [12]:
history = requests.post('http://localhost:8000/client/show_history', timeout=0.01, json={
    'client_token': client2['client_token'],
    'account_id': account2['info']['id'],
}).json()
history

{'history': [{'amount': 4000,
   'datetime': '2023-04-16T14:11:43.976502',
   'from': 'c88f3fda-8d1e-421c-b1dc-a11bf9f457cf',
   'id': '123521a0-6fce-446f-ba96-70b4d9fbdf49',
   'to': '3092d856-a0bd-464d-adc0-6c48146ab132'},
  {'amount': 1000,
   'datetime': '2023-04-16T14:11:51.036828',
   'from': '8b4a28f8-1209-4751-976d-db03799da425',
   'id': 'c6cc3983-c731-4706-adf9-d79d89b255d3',
   'to': '3092d856-a0bd-464d-adc0-6c48146ab132'}],
 'message': '',
 'status': 'ok'}

Откат последней транзакции:

In [13]:
requests.post('http://localhost:8000/superuser/cancel_transaction', timeout=0.01, json={
    'client_token': client2['client_token'],
    'account_id': account2['info']['id'],
    'transaction_id': history['history'][-1]['id']
}).json()

{'message': 'Canceled transaction c6cc3983-c731-4706-adf9-d79d89b255d3',
 'status': 'ok'}

Обновленная история у первого и второго клиента:

In [15]:
requests.post('http://localhost:8000/client/show_history', timeout=0.01, json={
    'client_token': client1['client_token'],
    'account_id': account1['info']['id'],
}).json()

{'history': [{'amount': -1000,
   'datetime': '2023-04-16T14:11:51.036828',
   'from': '3092d856-a0bd-464d-adc0-6c48146ab132',
   'id': 'c6cc3983-c731-4706-adf9-d79d89b255d3',
   'to': '8b4a28f8-1209-4751-976d-db03799da425'},
  {'amount': 1000,
   'datetime': '2023-04-16T14:12:15.865260',
   'from': '3092d856-a0bd-464d-adc0-6c48146ab132',
   'id': 'bbea27b3-5344-4d4e-b44a-c827edd7221d',
   'to': '8b4a28f8-1209-4751-976d-db03799da425'}],
 'message': '',
 'status': 'ok'}

In [16]:
requests.post('http://localhost:8000/client/show_history', timeout=0.01, json={
    'client_token': client2['client_token'],
    'account_id': account2['info']['id'],
}).json()

{'history': [{'amount': 4000,
   'datetime': '2023-04-16T14:11:43.976502',
   'from': 'c88f3fda-8d1e-421c-b1dc-a11bf9f457cf',
   'id': '123521a0-6fce-446f-ba96-70b4d9fbdf49',
   'to': '3092d856-a0bd-464d-adc0-6c48146ab132'},
  {'amount': 1000,
   'datetime': '2023-04-16T14:11:51.036828',
   'from': '8b4a28f8-1209-4751-976d-db03799da425',
   'id': 'c6cc3983-c731-4706-adf9-d79d89b255d3',
   'to': '3092d856-a0bd-464d-adc0-6c48146ab132'},
  {'amount': -1000,
   'datetime': '2023-04-16T14:12:15.865260',
   'from': '8b4a28f8-1209-4751-976d-db03799da425',
   'id': 'bbea27b3-5344-4d4e-b44a-c827edd7221d',
   'to': '3092d856-a0bd-464d-adc0-6c48146ab132'}],
 'message': '',
 'status': 'ok'}

Первый клиент отправил кредитные деньги, и транзакцию откатили. Соответственно у него сейчас 0:

In [17]:
requests.post('http://localhost:8000/client/show_accounts', timeout=0.01, json={
    'client_token': client2['client_token'],
}).json()

{'accounts': [{'balance': -4000,
   'id': 'c88f3fda-8d1e-421c-b1dc-a11bf9f457cf',
   'type': 'CashAccount'},
  {'balance': 4000,
   'id': '3092d856-a0bd-464d-adc0-6c48146ab132',
   'type': 'DepositAccount'}],
 'message': '',
 'status': 'ok'}